In [1]:
# Install necessary packages
!pip install yt-dlp langchain pydub langchain_community requests torch
!sudo apt-get install ffmpeg
!pip install youtube-transcript-api
!pip install gradio
!pip install google-generativeai
!pip install langchain-google-genai

!pip install chromadb
!pip install pypdf
!pip install sentence-transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.1/170.1 kB 2.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_6

In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain import PromptTemplate, LLMChain
from langchain_community.document_loaders import PyPDFLoader
from langchain.memory import ConversationBufferMemory
import gradio as gr
from langchain.schema import AIMessage, HumanMessage
import os
from youtube_transcript_api import YouTubeTranscriptApi
from langchain.schema import Document

In [7]:
def get_video_id(url):

    video_id = ""

    if "shorts/" in url:
        video_id = url.split("shorts/")[1].split("?")[0]

    elif "youtu.be/" in url:
        video_id = url.split("youtu.be/")[1].split("?")[0]

    elif "watch?v=" in url:
        video_id = url.split("watch?v=")[1].split("&")[0]

    return video_id
from langchain.schema import Document

def get_transcript(video_url, file_path='transcription.txt'):
    video_id = get_video_id(video_url)
    transcript_list = YouTubeTranscriptApi.get_transcript(video_id)

    text = " ".join([segment["text"] for segment in transcript_list]).strip()

    with open(file_path, 'w') as file:
        file.write(text)

    return file_path

In [8]:
from google.colab import userdata
api_key = userdata.get('GeminiKey')
if not api_key:
    raise ValueError("API key for Gemini is not set. Please set the GeminiProKey environment variable.")

In [11]:
chat_history = []

def generate_response(user_input, file_path=None):
    global chat_history
    if not user_input:
        return "No user input provided.", None
    if file_path and os.path.exists(file_path):
        with open(file_path, 'r') as file:
            transcription = file.read()

        text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
        documents = text_splitter.split_text(transcription)
        documents = [Document(page_content=chunk) for chunk in documents]

        try:
            if os.path.exists('db'):
                Chroma.delete("db")
            embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
            print(f"Embeddings initialized with model: 'sentence-transformers/all-MiniLM-L6-v2'")
            db = Chroma.from_documents(documents, embeddings)
        except Exception as e:
            return f"Error during vector store initialization: {str(e)}", file_path

        retrieved_results = db.similarity_search(user_input)

        history_context = "\n".join([f"User: {entry['user']}\nAssistant: {entry['assistant']}" for entry in chat_history])

        llm = ChatGoogleGenerativeAI(
            temperature=0.6,
            google_api_key=api_key,
            model='gemini-1.5-flash',
            max_output_tokens=2048,
            verbose=True,
        )

        prompt_template = PromptTemplate(
            input_variables=["context", "history", "question"],
            template="""You are a helpful assistant who answers questions based on the provided context and previous interactions.
                        Context: {context}
                        History: {history}
                        Question: {question}"""
        )
        llm_chain = LLMChain(llm=llm, prompt=prompt_template, verbose=True)

        output = llm_chain.run({"context": retrieved_results, "history": history_context, "question": user_input})

        chat_history.append({"user": user_input, "assistant": output})
        return output

    else:
        history_context = "\n".join([f"User: {entry['user']}\nAssistant: {entry['assistant']}" for entry in chat_history])

        llm = ChatGoogleGenerativeAI(
            temperature=0.6,
            google_api_key=api_key,
            model='gemini-1.5-flash',
            max_output_tokens=2048,
            verbose=True,
        )

        prompt_template = PromptTemplate(
            input_variables=["history", "question"],
            template="""You are a helpful assistant who answers questions based on previous interactions.
                        History: {history}
                        Question: {question}"""
        )
        llm_chain = LLMChain(llm=llm, prompt=prompt_template, verbose=True)

        output = llm_chain.run({"history": history_context, "question": user_input})
        chat_history.append({"user": user_input, "assistant": output})
        return output

In [13]:
import gradio as gr
def gradio_interface(youtube_url, question):
    file_path = None
    if youtube_url:
        file_path = get_transcript(youtube_url)
    response = generate_response(question, file_path)
    chat_history_text = "\n".join([f"User: {entry['user']}\nAssistant: {entry['assistant']}" for entry in chat_history])
    return response

iface = gr.Interface(
    fn=gradio_interface,
    inputs=[
        gr.Textbox(label="YouTube URL(Optional)"),
        gr.Textbox(label="Question")
    ],

    outputs="text",
    title="YouTube Video Chat",
    description="Enter a YouTube URL and ask questions about the video's content."
)

iface.launch(debug=True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://b45d100a27198eaa3b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)




> Entering new LLMChain chain...
Prompt after formatting:
You are a helpful assistant who answers questions based on previous interactions.
                        History: User: explain the story
Assistant: The story is about a fisherman and his wife who live in a humble hut near the sea. The wife is constantly complaining and never seems satisfied, always wanting more. One day, the fisherman catches an enchanted flounder who begs for his life. The fisherman, being kind, releases the flounder back into the sea.  

His wife is furious that he let go of the fish, believing he could have wished for a better house. She insists he go back and ask the flounder for a cottage. The fisherman reluctantly returns to the sea and finds the flounder waiting for him. The flounder agrees to grant his wife's wish for a cottage. 

                        Question: i am going to visit Vizag

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
You are a helpful assistant who a